In [2]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

In [3]:
# Crear una función para cambiar los 'none' a None en un .csv.

def remove_none (genre, start_page, end_page):
    #1- leer csv
    df = pd.read_csv(f'data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv')
    print (f"data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv opened")
    #2- localizar y sustituir 'none' por None
    df["birth_year"].replace("none", None, inplace=True)
    print ("csv updated")
    #3 guardar el csv con mismo nombre (sobreescribirlo)
    df.to_csv(f'data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv')
    print (f"data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv CREATED✅")

In [4]:
# execute the function to clean the files

genre = 'action'
start_page = 1
end_page = 5

for i in range(start_page, end_page):
    print(f"Iteracion {i}")
    remove_none (genre, i, i+1)

Iteracion 1
data/selenium_actors/list_action_range_1_2_sel_actor.csv opened
csv updated
data/selenium_actors/list_action_range_1_2_sel_actor.csv CREATED✅
Iteracion 2
data/selenium_actors/list_action_range_2_3_sel_actor.csv opened
csv updated
data/selenium_actors/list_action_range_2_3_sel_actor.csv CREATED✅
Iteracion 3
data/selenium_actors/list_action_range_3_4_sel_actor.csv opened
csv updated
data/selenium_actors/list_action_range_3_4_sel_actor.csv CREATED✅
Iteracion 4
data/selenium_actors/list_action_range_4_5_sel_actor.csv opened
csv updated
data/selenium_actors/list_action_range_4_5_sel_actor.csv CREATED✅


In [ ]:
# Function to insert 'list_oscars_BS' into the 'oscars' table.
def insert_oscars():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('data/oscars/list_oscars_BS.csv') 

        # SQL to insert data. You must indicate the name of the columns in the first parenthesis and in values enter the number of columns with %s
        sql_insert_query = """ INSERT INTO oscars (ceremony_year, best_movie, best_director, best_actor, best_actress) VALUES (%s, %s, %s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['ceremony_year'], row['best_movie'], row['best_director'], row['best_actor'], row['best_actress'])) 

        # Confirm changes with commit
        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [ ]:
# call the insert function for the oscars table
insert_oscars()

In [ ]:
# Function to insert the data into 'actor_details' table.
# IMPORTANT: it must be inserted before the 'actor_awards' table because to create its connections it needs to be already in the 'actor_awards' table.


def insert_actor_details(genre, start_page, end_page):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    mycursor = cnx.cursor()

    try: 
        df = pd.read_csv(f'data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv') # The CSV has an 'only' to distinguish temporary data.

        sql_insert_query = """ 
        INSERT INTO actor_details (actor_name, birth_year, known_for, actor_role, actor_awards)
        VALUES (%s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            actor_name = VALUES(actor_name),
            birth_year = VALUES(birth_year),
            known_for = VALUES(known_for),
            actor_role = VALUES(actor_role),
            actor_awards = VALUES(actor_awards)""" # We enter only one column value. It is temporary until we update it.

        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['actor_name'],row['birth_year'],row['known_for'],row['actor_role'],row['actor_awards'])) 

        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [ ]:
# call the insert function for the oscars table
genre = 'action'
start_page = 1
end_page = 2

for i in range(start_page, end_page):
    print(f"Iteracion {i}")
    insert_actor_details(genre, i, i+1)
    

In [ ]:
# Function to insert the data from 'actor_awards.csv' into the 'actor_awards' table. 
# It will not work if we don't have data in the 'actor_details' table.
def insert_actor_awards():
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    mycursor = cnx.cursor()

    try: 
        df = pd.read_csv('data/oscars/actor_awards.csv')
    
        sql_insert_query = """ INSERT INTO actor_awards (actor_name, ceremony_year, award_type) VALUES (%s, %s, %s)"""

        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['actor_name'], row['ceremony_year'], row['award_type'])) 

        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [ ]:
# call the insert function for the oscars table
insert_actor_awards()

In [ ]:
# insert data into relation_movie_oscars

def insert_relation_movie_oscars():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('data/oscars/relation_movie_oscars.csv')

        # SQL to insert data. You must indicate the name of the columns in the first parenthesis and in values enter the number of columns with %s
        sql_insert_query = """ INSERT INTO relation_movie_oscars (id_imdb, best_movie, ceremony_year) VALUES (%s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['id_imdb'], row['best_movie'], row['ceremony_year'])) 

        # Confirm changes with commit
        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝")

    except mysql.connector.Error as err:
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")


In [ ]:
# call the insert function for relation_movie_oscars

insert_relation_movie_oscars()